In [5]:
import pandas as pd
import numpy as np

In [7]:
totaldf = pd.read_csv('Data/totaldf.csv')

In [10]:
totaldf.columns

Index(['Unnamed: 0', 'title', 'link', 'release_date', 'been_sale',
       'orig_price', 'num_reviews', 'review_rating', 'num_languages', 'tags',
       'num_tags', 'genres', 'num_genres', 'publisher'],
      dtype='object')

In [12]:
totaldf.drop(columns=['Unnamed: 0'],inplace=True)

In [14]:
totaldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14981 entries, 0 to 14980
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          14981 non-null  object 
 1   link           14981 non-null  object 
 2   release_date   14939 non-null  object 
 3   been_sale      14981 non-null  int64  
 4   orig_price     14981 non-null  object 
 5   num_reviews    5652 non-null   object 
 6   review_rating  13913 non-null  object 
 7   num_languages  6890 non-null   object 
 8   tags           14980 non-null  object 
 9   num_tags       14980 non-null  float64
 10  genres         14981 non-null  object 
 11  num_genres     14981 non-null  float64
 12  publisher      14965 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 1.5+ MB


In [16]:
reviewsdf = totaldf[totaldf.num_reviews == 'HELP']

In [17]:
reviewsdf.shape

(124, 13)

In [18]:
totaldf.replace('HELP', np.nan, inplace=True)

In [30]:
totaldf[totaldf.num_reviews != np.nan]

,title,link,release_date,been_sale,orig_price,num_reviews,review_rating,num_languages,tags,num_tags,genres,num_genres,publisher
0,Sea of Thieves,https://store.steampowered.com/app/1172620/Sea...,"Jun 3, 2020",1,$39.99,"111,078",Very Positive,13,"['Adventure', 'Multiplayer', 'Open World', 'Pi...",20.0,"['Action', 'Adventure']",2.0,Xbox Game Studios
1,It Takes Two,https://store.steampowered.com/app/1426210/It_...,"Mar 26, 2021",0,$39.99,"11,813",Overwhelmingly Positive,12,"['Co-op', 'Adventure', '3D Platformer', 'Puzzl...",20.0,"['Action', 'Adventure', 'Hazelight']",3.0,Electronic Arts
2,Valheim,https://store.steampowered.com/app/892970/Valh...,"Feb 2, 2021",0,$19.99,"216,651",Overwhelmingly Positive,12,"['Open World Survival Craft', 'Online Co-Op', ...",20.0,"['Action', 'Adventure', 'Indie', 'RPG', 'Early...",6.0,Coffee Stain Publishing
3,Horizon Zero Dawn™ Complete Edition,https://store.steampowered.com/app/1151640/Hor...,"Aug 7, 2020",1,$49.99,"45,704",Mostly Positive,20,"['Adventure', 'Open World', 'Action', 'Female ...",20.0,"['Action', 'Adventure', 'RPG']",3.0,"PlayStation Mobile, Inc."
4,Phasmophobia,https://store.steampowered.com/app/739630/Phas...,"Sep 18, 2020",0,$13.99,"256,136",Overwhelmingly Positive,25,"['Horror', 'Online Co-Op', 'Multiplayer', 'Psy...",20.0,[],0.0,Kinetic Games
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14976,Dinosaur Hunt - Dragon Hunter Expansion Pack,https://store.steampowered.com/app/408102/Dino...,"Dec 18, 2015",0,$0.99,NaN,Mostly Positive,NaN,"['Adventure', 'Indie', 'Simulation']",3.0,"['Action', 'Adventure', 'Indie', 'Simulation']",4.0,Racing Bros
14977,Dinosaur Hunt - Giant Spiders Hunter Expansion...,https://store.steampowered.com/app/431191/Dino...,"Mar 16, 2016",0,$0.99,NaN,Mixed,NaN,"['Adventure', 'Indie', 'Simulation']",3.0,"['Action', 'Adventure', 'Indie', 'Simulation']",4.0,Racing Bros
14978,Dinosaur Hunt - Brontosaurus Expansion Pack,https://store.steampowered.com/app/434942/Dino...,"Mar 16, 2016",0,$0.99,NaN,Mixed,NaN,"['Adventure', 'Indie', 'Simulation', 'Action']",4.0,"['Action', 'Adventure', 'Indie', 'Simulation']",4.0,Racing Bros
14979,"Dinosaur Hunt - Vampires, Gargoyles, Mutants H...",https://store.steampowered.com/app/431190/Dino...,"Jan 22, 2016",0,$0.99,NaN,Mostly Positive,NaN,"['Adventure', 'Indie', 'Simulation']",3.0,"['Action', 'Adventure', 'Indie', 'Simulation']",4.0,Racing Bros


In [26]:
totaldf[totaldf.num_reviews>1]

TypeError: '>' not supported between instances of 'str' and 'int'

In [38]:
#reviews_df = totaldf.dropna(axis=0, subset=[totaldf.num_reviews])


In [40]:
droppeddf = totaldf.dropna()

In [41]:
droppeddf.shape

(2101, 13)

In [44]:
int(droppeddf.num_reviews)

TypeError: cannot convert the series to <class 'int'>

In [46]:
totaldf.num_reviews.astype('float')

ValueError: could not convert string to float: '111,078'

In [69]:
def converter (str):
    str.replace(',', '')
    

In [50]:
totaldf.loc[1:3,'num_reviews']

1     11,813
2    216,651
3     45,704
Name: num_reviews, dtype: object

In [52]:
strang = '11,813'
strang

'11,813'

In [58]:
strang.replace(',','')

'11813'

In [71]:
totaldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14981 entries, 0 to 14980
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          14981 non-null  object 
 1   link           14981 non-null  object 
 2   release_date   14939 non-null  object 
 3   been_sale      14981 non-null  int64  
 4   orig_price     14981 non-null  object 
 5   num_reviews    5528 non-null   object 
 6   review_rating  13908 non-null  object 
 7   num_languages  6744 non-null   object 
 8   tags           14980 non-null  object 
 9   num_tags       14980 non-null  float64
 10  genres         14981 non-null  object 
 11  num_genres     14981 non-null  float64
 12  publisher      14964 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 1.5+ MB


In [72]:
totaldf.num_reviews.apply(converter)

AttributeError: 'float' object has no attribute 'replace'

In [123]:
list1=[]
for nums in totaldf.num_reviews:
    list1.append(nums)


In [124]:
len(list1)

14981

In [128]:
list2 = []
for items in list1:
        list2.append(str(items.replace(',','')))

In [126]:
list1[0].replace(',','')

'111078'

In [130]:
totaldf.replace('nan',np.nan,inplace=True)

In [129]:
list2

['111078',
 '11813',
 '216651',
 '45704',
 '256136',
 '23944',
 '598684',
 '38064',
 '8397',
 '73464',
 '74220',
 '87734',
 '446261',
 'nan',
 'nan',
 '148259',
 '235',
 '37643',
 '60517',
 '346111',
 'nan',
 '7953',
 'nan',
 '407256',
 '174885',
 '9324',
 '260518',
 '356033',
 '111244',
 'nan',
 'nan',
 '302018',
 'nan',
 '534568',
 'nan',
 '21377',
 'nan',
 'nan',
 'nan',
 '138362',
 '263559',
 'nan',
 'nan',
 'nan',
 '5372',
 'nan',
 '721278',
 'nan',
 '66011',
 'nan',
 '43581',
 '3203',
 '55841',
 'nan',
 '69968',
 '163092',
 'nan',
 '84207',
 '418832',
 '32968',
 'nan',
 '12411',
 '1469',
 '5348',
 '40998',
 '897872',
 '40082',
 '154',
 '98920',
 '189749',
 'nan',
 'nan',
 '137725',
 '23181',
 '684935',
 'nan',
 '706',
 'nan',
 '21668',
 'nan',
 '5887',
 '5008',
 '119296',
 'nan',
 '84494',
 '479077',
 '96979',
 '159604',
 '499',
 '5245',
 'nan',
 '912',
 '18590',
 '19814',
 '17350',
 '1697393',
 'nan',
 'nan',
 '4296',
 '40674',
 '50151',
 '24066',
 '99684',
 '24438',
 '91488',
 

In [96]:
totaldf.num_reviews.as_numeric()

AttributeError: 'Series' object has no attribute 'as_numeric'

In [98]:
totaldf[totaldf.num_reviews.dropna()]

KeyError: "None of [Index(['111,078', '11,813', '216,651', '45,704', '256,136', '23,944',\n       '598,684', '38,064', '8,397', '73,464',\n       ...\n       '15', '10', '16', '487', '193', '155', '960', '56', '65', '47'],\n      dtype='object', length=5528)] are in the [columns]"

In [108]:
totaldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14981 entries, 0 to 14980
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          14981 non-null  object 
 1   link           14981 non-null  object 
 2   release_date   14939 non-null  object 
 3   been_sale      14981 non-null  int64  
 4   orig_price     14981 non-null  object 
 5   num_reviews    5528 non-null   object 
 6   review_rating  13908 non-null  object 
 7   num_languages  6744 non-null   object 
 8   tags           14980 non-null  object 
 9   num_tags       14980 non-null  float64
 10  genres         14981 non-null  object 
 11  num_genres     14981 non-null  float64
 12  publisher      14964 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 1.5+ MB


In [114]:
#totaldf.dropna(subset=totaldf.num_reviews)

In [119]:
totaldf.num_reviews.value_counts

<bound method IndexOpsMixin.value_counts of 0        111,078
1         11,813
2        216,651
3         45,704
4        256,136
          ...   
14976        NaN
14977        NaN
14978        NaN
14979        NaN
14980        NaN
Name: num_reviews, Length: 14981, dtype: object>

In [122]:
totaldf.num_reviews = totaldf.num_reviews.astype(str)

In [134]:
totaldf.num_reviews = totaldf.num_reviews.astype(int)

ValueError: invalid literal for int() with base 10: '111,078'

In [145]:
totaldf.drop(totaldf.loc[totaldf['num_reviews'] == np.nan].index, inplace=True)

In [144]:
totaldf.shape

(14981, 13)

In [137]:
# cannot for the life of me figure out how to convert these data types or drop nulls

In [146]:
np.nan in totaldf.num_reviews

False

In [148]:
totaldf.num_reviews.value_counts

<bound method IndexOpsMixin.value_counts of 0        111,078
1         11,813
2        216,651
3         45,704
4        256,136
          ...   
14976        NaN
14977        NaN
14978        NaN
14979        NaN
14980        NaN
Name: num_reviews, Length: 14981, dtype: object>